1) Define a function that realizes the Cartesian product between vectors and create the exercise matrices (strategy matrix, probability matrix, and payoff matrix):

In [22]:
using SymPy

# Cartesian product
function s_matrix(x)
    S = vec(collect(Base.product(reverse(x)...))) 
    S_M = []
    for i = 1:length(S[1])
        d = [y[i] for y in S]
        if i == 1 S_M = [d; ] else S_M = [S_M d] end
    end
    S_M = reverse(S_M, dims = 2)
    return S_M
end

s_matrix (generic function with 1 method)

2) Create the strategy matrix:

In [23]:
a1 = [Sym("a_$i^$j") for i=1 for j=["c","nc"]]
a2 = [Sym("a_$i^$j") for i=2 for j=["c","nc"]]
A = [a1 , a2]
S = s_matrix(A) 

4×2 Matrix{Sym}:
  a_1__c   a_2__c
  a_1__c  a_2__nc
 a_1__nc   a_2__c
 a_1__nc  a_2__nc

3) Create the probability matrix:

In [24]:
p1 = [Sym("alpha_$i^$j") for i=1 for j=["c","nc"]]
p2 = [Sym("alpha_$i^$j") for i=2 for j=["c","nc"]]
P = [p1 , p2]
P = s_matrix(P)

4×2 Matrix{Sym}:
  α_1__c   α_2__c
  α_1__c  α_2__nc
 α_1__nc   α_2__c
 α_1__nc  α_2__nc

4) Create the payoff matrix:

In [25]:
x = symbols("x", integer=true, positive = true, nonzero = true)
U = [2*x -x; -x 2*x;-x 2*x;2*x -x] 

4×2 Matrix{Sym}:
 2⋅x   -x
  -x  2⋅x
  -x  2⋅x
 2⋅x   -x

5) We will use the created matrices to build a system of equations. The solution of this system of equations will give us the solution of games with mixed strategies.  

The first set of equations of our exercise comes from the fact that the sum of the probabilities associated with the actions of the same player must be equal to 1. In other words, if a player (i) can only perform the actions $a^{c}_{ i}$ or $a^{nc}_{i}$, and each of these actions has different rates of being performed by this player ($\alpha^{c}_{i}$ or $\alpha^{nc }_{i}$, respectively), so the chance of one or the other action occurring must be equal to 1. This logic would hold, regardless of the number of actions of player i.

$$\alpha^{c}_{i} + \alpha^{nc}_{i} = 1$$



In [26]:
# The sum of the probabilities of a player must be equal to one.
function sum_prob(x)
    eqn = []
    for i = 1:size(x, 2) 
        f = Eq(sum(unique(x[:,i])),1)
        if i == 1 eqn = [f; ] else eqn = [eqn f] end
    end
    return eqn
end
eq1 = sum_prob(P)

1×2 Matrix{Sym}:
 α_1__c + α_1__nc = 1  α_2__c + α_2__nc = 1

6) The following equations that will make up our system of equations depend on the expected utility of each player. So the next step is to calculate such utilities.

In [27]:
#Hadamard product(Find the product between elements of the same line).
# Each line is a different conditional probability.
function hadamard_product(x)
    ncol = size(x, 2)
    d = []
    for i = 1:(ncol - 1)
        if i == 1 d = (x[:,i] .* x[:,(i+1)]) else d = (x[:,i+1] .* d) end   
    end    
    return d
end

S_P = hadamard_product(P)
#Find the expected payoff for each payer.
eq2 = S_P.T * U

1×2 Matrix{Sym}:
 2⋅α_1__c⋅α_2__c⋅x - α_1__c⋅α_2__nc⋅x - α_1__nc⋅α_2__c⋅x + 2⋅α_1__nc⋅α_2__nc⋅x  …  -α_1__c⋅α_2__c⋅x + 2⋅α_1__c⋅α_2__nc⋅x + 2⋅α_1__nc⋅α_2__c⋅x - α_1__nc⋅α_2__nc⋅x

7) Player (i) defines the ability to perform a specific action. Furthermore, that definition is based on maximizing its expected payoff. In other words, what is the probability $\alpha_{i}^{c}$ that player (i) must attribute to the performance of the action $a_{i}^{c}$ that will bring him more payoff? The answer lies in the player's first-order conditions and their usefulness. The players' first-order conditions form the last set of equations we need to solve our game.

In [28]:
#Find first-order conditions for each player.
function foc(eq1,eq2,P)
    eq2_aux = []
    eq3 = Vector{Sym}()
    for pl in 1:size(P,2) # Players
        var = unique(P[:,pl])
        for i in 1:(size(var)[1]-1)
            for j in var 
                if j != var[i] 
                    eq2_aux = eq2[pl].subs.(j,solve(eq1[pl],j))
                    eq = Eq(diff.(eq2_aux[1],var[i]),0)
                    append!(eq3, eq)
                end
            end
        end
    end
    return eq3
end


eq3 = foc(eq1,eq2,P)

2-element Vector{Sym}:
  3⋅α_2__c⋅x - 3⋅α_2__nc⋅x = 0
 -3⋅α_1__c⋅x + 3⋅α_1__nc⋅x = 0

8) finally we group our equations and solve our system of equations algebraically. 

In [29]:
#equations of the sistem
eq4 = append!(vec(eq1), eq3)

4-element Vector{Sym}:
          α_1__c + α_1__nc = 1
          α_2__c + α_2__nc = 1
  3⋅α_2__c⋅x - 3⋅α_2__nc⋅x = 0
 -3⋅α_1__c⋅x + 3⋅α_1__nc⋅x = 0

In [30]:
vars = unique(P) # system variables 
sol = solve(eq4,vars)
sol = [Eq(var[1],var[2]) for var in sol] 

4-element Vector{Sym}:
 α_2__nc = 1/2
  α_1__c = 1/2
 α_1__nc = 1/2
  α_2__c = 1/2